In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import textblob
import tensorflow as tf

In [23]:
with open("yelp_reviews_sample.json") as f:
    records = [json.loads(x) for x in f]
df = pd.DataFrame.from_dict(records)
df['datetime'] = pd.to_datetime(df['date'])

In [35]:
df.sort_values('funny').tail(10)['text'].iloc[0]

"4.5/5 stars\n\nI called Jene's Tropical to find a fig tree. We wanted a Black Mission Fig but there were none locally. I connected to an answering machine. I don't usually leave messages because I don't like waiting but I  left a message and they called back that day. \n\nThey did have small fig trees in stock.  We made the trip across the bridge to Jene's. Parking is on a small street with homes on one side so it can be tight. The store is good sized and filled with trees, smaller plants, pots and outdoor decor. We were greeted and shown the fig trees. They were small but they had more coming in the next week.  We took a very healthy fig tree, some organic fertilizer and an organic fruit & citrus fertilizer for later to produce more fruit and help the tree resist disease, insects and drought. \n\nJene's has all organic plants, trees and supplies. I recommend this store and will definitely go back. Their trees and plants were healthy and all the materials were of great quality. The pr

In [36]:
# need to pre-process text
# remove punctuation, filler words, new line characters

In [37]:
# need to split data into test and training

In [38]:
# need to train tf model 